In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
import rfpy
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada) ;  arquivos

(#3) [Path('C:/Users/rsilva/Downloads/entrada/201201_T153421_OneThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/201201_T154509_MultiplesThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/rfeye002304_SLMA_bimestral_PEAK_200829_234902.bin')]

In [ ]:
one_tid = arquivos[-1]
mult_tid = arquivos[1]

On a normal conda environment the time of the following cell was
```python
%%timeit
one = process_bin(one_tid)
```
1.22 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %%timeit
# one = parse_bin(one_tid)

On a normal conda environment the time of the following cell was
```python
%%timeit
multi = process_bin(mult_tid)
```
1.29 s ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %%timeit
# multi = parse_bin(mult_tid)

In [ ]:
#one = parse_bin(one_tid)

In [ ]:
#one.keys()

In [ ]:
#one['blocos']

In [ ]:
multi = parse_bin(mult_tid)

In [ ]:
multi.keys()

dict_keys(['file_version', 'string', 'blocos'])

In [ ]:
multi['file_version']

22

In [ ]:
multi['string']

'CRFS DATA FILE V022'

In [ ]:
multi['blocos']

defaultdict(fastcore.foundation.L,
            {(21,
              0): (#1) [((36, 123), <rfpy.blocks.DType21 object at 0x000001DB336826C8>)],
             (41,
              0): (#2) [((128, 203), <rfpy.blocks.DType41 object at 0x000001DB33682548>),((208, 335), <rfpy.blocks.DType41 object at 0x000001DB33682C08>)],
             (24,
              5): (#1) [((340, 375), <rfpy.blocks.DType24 object at 0x000001DB33682A08>)],
             (24,
              10): (#1) [((380, 427), <rfpy.blocks.DType24 object at 0x000001DB33682408>)],
             (24,
              20): (#1) [((432, 487), <rfpy.blocks.DType24 object at 0x000001DB33682288>)],
             (24,
              30): (#1) [((492, 547), <rfpy.blocks.DType24 object at 0x000001DB33682088>)],
             (24,
              40): (#1) [((552, 607), <rfpy.blocks.DType24 object at 0x000001DB33680A48>)],
             (24,
              50): (#1) [((612, 667), <rfpy.blocks.DType24 object at 0x000001DB33680948>)],
             (24,
      

In [ ]:
x = multi['blocos'][(21,0)][0]

In [ ]:
x

((36, 123), <rfpy.blocks.DType21 at 0x1db336826c8>)

In [ ]:
bin_data = x[1].data ; bin_data

b'rfeye002292\x00\x00\x00\x00\x00\x18\x00\x00\x00CRFS default unit info\x00\x00\x14\x00\x00\x00CRFS default method\x00\x00\x00\x00'

In [ ]:
len(x[1].data)

71

In [ ]:
getattrs(x[1])

{'data_size': 72,
 'file_number': 0,
 'hostname': 'rfeye002292',
 'method': 'CRFS default method',
 'thread_id': 0,
 'type': 21,
 'unit_info': 'CRFS default unit info'}

In [ ]:
x[1]._text1_len

24

In [ ]:
x[1]._text2_len

20

In [ ]:
x[1].STOP

72

In [ ]:
x[0]

(36, 123)

In [ ]:
with open(mult_tid, mode='rb') as bfile:
    # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
    body = bfile.read()

In [ ]:
block = create_base_block(body[slice(*x[0])])

In [ ]:
test_eq(bin_data, block.data)

In [ ]:
getattrs(x)

In [ ]:
L(x.data[-100:])

In [ ]:
b1 = multi['blocos'][(63,40)]

In [ ]:
index = b1.itemgot(0) ; index

(#1281) [(41928, 44079),(119544, 121695),(197160, 199311),(274776, 276927),(352392, 354543),(430008, 432159),(507624, 509775),(585240, 587391),(662856, 665007),(744784, 746935)...]

In [ ]:
index = index.map(lambda x: dict(zip(('start_byte', 'stop_byte'), x))) ; index

(#1281) [{'start_byte': 41928, 'stop_byte': 44079},{'start_byte': 119544, 'stop_byte': 121695},{'start_byte': 197160, 'stop_byte': 199311},{'start_byte': 274776, 'stop_byte': 276927},{'start_byte': 352392, 'stop_byte': 354543},{'start_byte': 430008, 'stop_byte': 432159},{'start_byte': 507624, 'stop_byte': 509775},{'start_byte': 585240, 'stop_byte': 587391},{'start_byte': 662856, 'stop_byte': 665007},{'start_byte': 744784, 'stop_byte': 746935}...]

In [ ]:
blocos = multi['blocos'][(63,40)].itemgot(1).map(getattrs) ; blocos

(#1281) [{'bw': 40, 'data_size': 2136, 'gerror': -1, 'gflags': -1, 'group_id': 0, 'id_antenna': 0, 'level_offset': -20, 'n_agc': 7, 'n_tunning': 7, 'npad': 1, 'npoints': 2048, 'num_meas': 1, 'processing': 'average', 'rbw': 19540, 'spent_time_microsecs': 5570, 'start_channel': 0, 'start_mega': 70, 'start_mili': 0, 'step': 0.019540791402051783, 'stop_channel': 0, 'stop_mega': 110, 'stop_mili': 0, 'thread_id': 40, 'tunning_info': {'60MHz': 'completed without error', '70MHz': 'completed without error', '80MHz': 'completed without error', '90MHz': 'completed without error', '100MHz': 'completed without error', '110MHz': 'completed without error', '120MHz': 'completed without error'}, 'type': 63, 'unit': '%', 'wallclock_datetime': datetime.datetime(2020, 12, 1, 15, 45, 9, 816463)},{'bw': 40, 'data_size': 2136, 'gerror': -1, 'gflags': -1, 'group_id': 0, 'id_antenna': 0, 'level_offset': -20, 'n_agc': 7, 'n_tunning': 7, 'npad': 1, 'npoints': 2048, 'num_meas': 1, 'processing': 'average', 'rbw': 

In [ ]:
meta = b1.map(lambda item: {**dict(zip(('start_byte', 'stop_byte'), item[0])), **getattrs(item[1])}) ; meta

(#1281) [{'start_byte': 41928, 'stop_byte': 44079, 'bw': 40, 'data_size': 2136, 'gerror': -1, 'gflags': -1, 'group_id': 0, 'id_antenna': 0, 'level_offset': -20, 'n_agc': 7, 'n_tunning': 7, 'npad': 1, 'npoints': 2048, 'num_meas': 1, 'processing': 'average', 'rbw': 19540, 'spent_time_microsecs': 5570, 'start_channel': 0, 'start_mega': 70, 'start_mili': 0, 'step': 0.019540791402051783, 'stop_channel': 0, 'stop_mega': 110, 'stop_mili': 0, 'thread_id': 40, 'tunning_info': {'60MHz': 'completed without error', '70MHz': 'completed without error', '80MHz': 'completed without error', '90MHz': 'completed without error', '100MHz': 'completed without error', '110MHz': 'completed without error', '120MHz': 'completed without error'}, 'type': 63, 'unit': '%', 'wallclock_datetime': datetime.datetime(2020, 12, 1, 15, 45, 9, 816463)},{'start_byte': 119544, 'stop_byte': 121695, 'bw': 40, 'data_size': 2136, 'gerror': -1, 'gflags': -1, 'group_id': 0, 'id_antenna': 0, 'level_offset': -20, 'n_agc': 7, 'n_tunn

In [ ]:
df = pd.DataFrame(meta).set_index('wallclock_datetime') ; df.head()

,start_byte,stop_byte,bw,data_size,gerror,gflags,group_id,id_antenna,level_offset,n_agc,...,start_mega,start_mili,step,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit
wallclock_datetime,,,,,,,,,,,,,,,,,,,,,
2020-12-01 15:45:09.816463,41928,44079,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,40,"{'60MHz': 'completed without error', '70MHz': ...",63,%
2020-12-01 15:45:10.318062,119544,121695,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,40,"{'60MHz': 'completed without error', '70MHz': ...",63,%
2020-12-01 15:45:10.817929,197160,199311,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,40,"{'60MHz': 'completed without error', '70MHz': ...",63,%
2020-12-01 15:45:11.316203,274776,276927,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,40,"{'60MHz': 'completed without error', '70MHz': ...",63,%
2020-12-01 15:45:11.810011,352392,354543,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,40,"{'60MHz': 'completed without error', '70MHz': ...",63,%


In [ ]:
%%time
metadata = export_bin_meta(multi['blocos'])

Wall time: 17.6 s


In [ ]:
metadata[-1][1]

,start_byte,stop_byte,bw,data_size,gerror,gflags,group_id,id_antenna,level_offset,n_agc,...,start_mili,step,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit,wallclock_datetime
0,675792,677943,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:45:13.817044
1,1456264,1458415,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:45:18.812037
2,2236736,2238887,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:45:23.818756
3,3017208,3019359,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:45:28.842880
4,3797680,3799831,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:45:33.842036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,96673848,96675999,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:55:30.884885
124,97454320,97456471,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:55:35.892739
125,98234792,98236943,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:55:40.917794
126,99015264,99017415,40,2136,-1,-1,0,0,-20,7,...,0,0.019541,0,110,0,100,"(60MHz-completed without error, 70MHz-complete...",63,%,2020-12-01 15:55:45.915822


In [ ]:
b2 = multi['blocos'][(63,50)].map(getattrs)

In [ ]:
df2 = pd.DataFrame(b2).set_index('datetime_stamp') ; df2.head()

In [ ]:
b3 = one['blocos'][(63,20)].map(getattrs)

In [ ]:
df3 = pd.DataFrame(b3).set_index('datetime_stamp') ; df3.head()

In [ ]:
dfs = [pd.DataFrame(v.map(getattrs)) for v in one['blocos'].values() ]

In [ ]:
dfs[0]

In [ ]:
dfs[1]

In [ ]:
dfs[2]

In [ ]:
dfs[3]

In [ ]:
dfs[4]

In [ ]:
dfs[5]

In [ ]:
dfs[6]

In [ ]:
dfs[7]

In [ ]:
dfs[8]

In [ ]:
dfs[9]

In [ ]:
dfs[10]

In [ ]:
dfs[11]